In [195]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import bs4 as bs
import pandas as pd

In [196]:
service = Service(ChromeDriverManager().install())


In [197]:
marca = ['xpro nutrition']
ncl = ['']

In [198]:
# basica radical 


browser = webdriver.Chrome(service=service)
browser.get("https://busca.inpi.gov.br")
time.sleep(2)
browser.find_element(By.XPATH,'//*[@id="principal"]/form/table/tbody/tr[6]/td/font/b/font/a').click();
time.sleep(2)



browser.get('https://busca.inpi.gov.br/pePI/servlet/LoginController?action=login')

# segue a trilha para a pagina de pesuisa e seleciona basica por radical
browser.find_element(By.CSS_SELECTOR, 'area[href="/pePI/jsp/marcas/Pesquisa_num_processo.jsp"]').click();
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[1]/td/font/a[1]').click();
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[4]/td[2]/input[2]').click();

# insere a marca e ncl na caixa de pesquisa
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[6]/td[2]/font/input').send_keys(marca);
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[8]/td[2]/font[1]/input').send_keys(ncl);
time.sleep(1)

# seleciona o numero maximo de reistros por tela (100)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[10]/td[2]/font/select/option[1]').click()
time.sleep(1)
#pesquisa
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[11]/td/font/input[1]').click()



In [199]:
# obtendo a estrutura html da pagina e transformando em um objeto 'soup'
html = browser.page_source
soup = bs.BeautifulSoup(html, 'html.parser')

In [200]:
# Colunas 

# td[0] = num_registro
# td[1] = dta_prioridade
# td[2] = marca
# td[3] = status
# td[4] = titular
# td[5] = ncl


In [201]:
mtx_itens = []
itens = []

In [202]:
tb = soup.find_all('table')[2].find('tbody').find_all('tr')

In [203]:
# para saber quantos marcas a consulta retornou, basta verificar o tamanho da variavel
# que guarda todas as linhas e contar, subtraindo o header e footer
len(tb) - 2

1

In [204]:
tb

[<tr>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"><font class="normal"><b>Número</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"><font class="normal"><b>Prioridade</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="18" width="5%"><font class="normal"> </font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="25%"><font class="normal"><b>Marca</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="18" width="5%"><font class="normal"> </font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"> <font class="normal"><b>Situação</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="30%"><font class="normal"><b>Titular</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="20%"><font class="normal"><b>Classe</b></font></td>
 </tr>,
 <tr bgcolor="#E0E0E0" class="normal">
 <td align="center">
 <font class="normal">
 <a class="visitado" href="/pePI/servlet/MarcasServle

In [205]:
for i in (range(len(tb) - 1)):
    itens = []
    for j in (range(len(tb[i].find_all('td')))):
        itens.append( tb[i].find_all('td')[j].get_text().strip() )
    mtx_itens.append(itens)




In [206]:
mtx_itens

[['Número', 'Prioridade', '', 'Marca', '', 'Situação', 'Titular', 'Classe'],
 ['930686195',
  '06/06/2023',
  '',
  'XPRO NUTRITION',
  '',
  'Aguardando exame de mérito',
  'CR NUTRITION SUPLEMENTOS EIRELI',
  'NCL(12) 05']]

In [207]:
header = mtx_itens[0]  # Primeira linha da matriz
data = mtx_itens[1:]  # Resto dos dados da matriz

In [208]:
df = pd.DataFrame(data=data, columns=header)

In [209]:
df.select()

,Número,Prioridade,,Marca,,Situação,Titular,Classe
0,930686195,06/06/2023,,XPRO NUTRITION,,Aguardando exame de mérito,CR NUTRITION SUPLEMENTOS EIRELI,NCL(12) 05
